In [1]:
import cv2
import tensorflow as tf
import argparse
import numpy as np
import os
import pdb
import time
import matplotlib.pyplot as plt
import sys
print(tf.__version__)
import scipy.io as sio
import math

############### Global Parameters ###############
# path
video_list  = sio.loadmat('trainlist_ix2.mat')
test_list = sio.loadmat('testlist_ix2.mat')

train_path = './experiments/ixmas/rgb_data_split2/train/'

train_path_2 = './experiments/ixmas/flow_data_split2/train/'

test_path = './experiments/ixmas/rgb_data_split2/test/'

test_path_2 = './experiments/ixmas/flow_data_split2/test/'

train_path_3 = './experiments/ixmas/flowt_data_split2/train/'

test_path_3 = './experiments/ixmas/flowt_data_split2/test/'

# Model Path
default_model_path = './multi_model/ixmas_fusion/final_model'
save_path = './multi_model/ixmas_fusion/'

# Output Path
output_path = './ixmas_fusion_res/'

# Train Test Split
train_num = video_list['video'][0].shape[0]-1
test_num = test_list['video'][0].shape[0]-1

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
tf.test.is_gpu_available()

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


1.10.0


True

In [2]:
video_list ['video'][0][1][0]

array([u'chiara3_01_check-watch_cam2_frames_0029_0093'], dtype='<U44')

In [3]:
############## Train Parameters #################
# Parameters
learning_rate = 0.00002
n_epochs = 100
batch_size = 1
# Network Parameters
n_input = 1024 # input dimension
n_hidden = 1024 # hidden layer num of multi_head
n_classes = 12 # has 51 classes
n_frames = 32

In [4]:
def squared_dist(A, B):
    assert A.shape.as_list() == B.shape.as_list()

    row_norms_A = tf.reduce_sum(tf.square(A), axis=1)
    row_norms_A = tf.reshape(row_norms_A, [-1, 1])  # Column vector.

    row_norms_B = tf.reduce_sum(tf.square(B), axis=1)
    row_norms_B = tf.reshape(row_norms_B, [1, -1])  # Row vector.

    return row_norms_A - 2 * tf.matmul(A, tf.transpose(B)) + row_norms_B

In [5]:
def normalize(inputs, 
              epsilon = 1e-8,
              scope="ln",
              reuse=None):
    '''Applies layer normalization.
    
    Args:
      inputs: A tensor with 2 or more dimensions, where the first dimension has
        `batch_size`.
      epsilon: A floating number. A very small number for preventing ZeroDivision Error.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
      
    Returns:
      A tensor with the same shape and data dtype as `inputs`.
    '''
    with tf.variable_scope(scope, reuse=reuse):
        inputs_shape = inputs.get_shape()
        params_shape = inputs_shape[-1:]
    
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
        beta= tf.Variable(tf.zeros(params_shape))
        gamma = tf.Variable(tf.ones(params_shape))
        normalized = (inputs - mean) / ( (variance + epsilon) ** (.5) )
        outputs = gamma * normalized + beta
        
    return outputs

In [6]:
def multihead_attention(queries, 
                        keys,
                        wkeys, 
                        num_units=None, 
                        num_heads=8, 
                        dropout_rate=0,
                        is_training=True,
                        causality=False,
                        trainable=True,
                        scope="multihead_attention", 
                        reuse=None):
    '''Applies multihead attention.
    
    Args:
      queries: A 3d tensor with shape of [N, T_q, C_q].
      keys: A 3d tensor with shape of [N, T_k, C_k].
      num_units: A scalar. Attention size.
      dropout_rate: A floating point number.
      is_training: Boolean. Controller of mechanism for dropout.
      causality: Boolean. If true, units that reference the future are masked. 
      num_heads: An int. Number of heads.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
        
    Returns
      A 3d tensor with shape of (N, T_q, C)  
    '''
    with tf.variable_scope(scope, reuse=reuse):
        # Set the fall back option for num_units
        if num_units is None:
            num_units = queries.get_shape().as_list[-1]
        
        # Linear projections
        Q = tf.contrib.layers.fully_connected(queries, num_units,trainable=trainable) # (N, T_q, C)
        K = tf.contrib.layers.fully_connected(keys, num_units,trainable=trainable ) # (N, T_k, C)
        V = tf.contrib.layers.fully_connected(keys, num_units,trainable=trainable) # (N, T_k, C)
        
        Q1 = tf.reshape(Q,(1,n_frames,num_units))
        K1 = tf.reshape(K,(1,n_frames,num_units))        
        V1 = tf.reshape(V,(1,n_frames,num_units))
        
        
        # Split and concat
        Q_ = tf.concat(tf.split(Q1, num_heads, axis = 2),axis =0) # (h*N, T_q, C/h) 
        K_ = tf.concat(tf.split(K1, num_heads, axis = 2),axis =0) # (h*N, T_k, C/h) 
        V_ = tf.concat(tf.split(V1, num_heads, axis = 2),axis =0) # (h*N, T_k, C/h) 

        
        # Multiplication
        #outputs = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1])) # (h*N, T_q, T_k)
        output_1 = tf.matmul(Q_, wkeys) # (h*N, T_q, T_k)
        outputs  = tf.matmul(output_1, tf.transpose(K_, [0, 2, 1])) # (h*N, T_q, T_k)

        
        # Scale
        outputs = outputs / (K_.get_shape().as_list()[-1] ** 0.5)

                
        # Key Masking
        key_masks = tf.sign(tf.abs(tf.reduce_sum(keys, axis=-1))) # (N, T_k)
        key_masks = tf.tile(key_masks, [num_heads, 1]) # (h*N, T_k)
        key_masks = tf.tile(tf.expand_dims(key_masks, 1), [1, tf.shape(queries)[1], 1]) # (h*N, T_q, T_k)
       # 
        paddings = tf.ones_like(outputs)*(-2**32+1)
        outputs = tf.where(tf.equal(key_masks, 0), paddings, outputs) # (h*N, T_q, T_k)
          
        # Causality = Future blinding
        if causality:
            diag_vals = tf.ones_like(outputs[0, :, :]) # (T_q, T_k)
            tril = tf.contrib.linalg.LinearOperatorTriL(diag_vals).to_dense() # (T_q, T_k)
            masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(outputs)[0], 1, 1]) # (h*N, T_q, T_k)
  # 
            paddings = tf.ones_like(masks)*(-2**32+1)
            outputs = tf.where(tf.equal(masks, 0), paddings, outputs) # (h*N, T_q, T_k)
  
        # Activation
        outputs = tf.nn.softmax(outputs) # (h*N, T_q, T_k)
        matt    = outputs
        
        
        # Query Masking
        query_masks = tf.sign(tf.abs(tf.reduce_sum(queries, axis=-1))) # (N, T_q)
        query_masks = tf.tile(query_masks, [num_heads, 1]) # (h*N, T_q)
        query_masks = tf.tile(tf.expand_dims(query_masks, -1), [1, 1, tf.shape(keys)[1]]) # (h*N, T_q, T_k)
        outputs *= query_masks # broadcasting. (N, T_q, C)
          
        # Dropouts
        outputs = tf.contrib.layers.dropout(outputs, keep_prob=dropout_rate, is_training=tf.convert_to_tensor(is_training))
               
        # Weighted sum
        outputs = tf.matmul(outputs, V_) # ( h*N, T_q, C/h)
        
        # Restore shape
        outputs = tf.concat(tf.split(outputs, num_heads,axis = 0),axis =2 ) # (N, T_q, C)
              
        # Residual connection
        outputs += queries
              
        # Normalize
        outputs = normalize(outputs) # (N, T_q, C)
 
    return outputs, matt

In [7]:
def build_model():

    # tf Graph input
    x   = tf.placeholder("float", [None, n_frames , n_input])
    x2  = tf.placeholder("float", [None, n_frames , n_input])
    x3  = tf.placeholder("float", [None, n_frames , n_input])
    y   = tf.placeholder("float", [None, n_classes])
    # Define weights
    weights = {
        'out'    : tf.Variable(tf.random_normal([n_hidden, n_classes], mean=0.0, stddev=0.01)),
        'att_wk' : tf.Variable(tf.random_normal([8,n_input/8, n_input/8], mean=0.0, stddev=0.02),trainable=True),
        'att_wk2': tf.Variable(tf.random_normal([8,n_input/8, n_input/8], mean=0.0, stddev=0.02),trainable=True),
        'att_wk3': tf.Variable(tf.random_normal([8,n_input/8, n_input/8], mean=0.0, stddev=0.02),trainable=True)

    }
    biases = {
        'out': tf.Variable(tf.random_normal([n_classes], mean=0.0, stddev=0.01))


    }
    # Attention Weighting
#     m1 = tf.get_variable('m1',dtype=tf.float32, shape = [1])
#     m2 = tf.get_variable('m2',dtype=tf.float32, shape = [1])
#     m3 = tf.get_variable('m3',dtype=tf.float32, shape = [1])
#     m1 = tf.sigmoid(m1)
#     m2 = tf.sigmoid(m2)
#     m3 = tf.sigmoid(m3)
    # init loss 
    loss = 0.0  
    # Mask 
    regularizer = tf.contrib.layers.l1_regularizer(scale=0.0005)
    reg_term    = tf.contrib.layers.apply_regularization(regularizer,[weights['att_wk']])
    # Start creat graph
    X  = tf.reshape(x,(1,n_frames,n_input))
    X2 = tf.reshape(x2,(1,n_frames,n_input))
    X3 = tf.reshape(x3,(1,n_frames,n_input))
            
    # 1 rank approximation of attention weighting  
    wkeys1 = weights['att_wk']
    wkeys2 = weights['att_wk2']
    wkeys3 = weights['att_wk3']
        
        
    # Multi attention   
    multi_att, matt =  multihead_attention(queries=X, keys=X, wkeys = wkeys1, num_units=n_input, 
                                                        num_heads=8, 
                                                        dropout_rate=0.9,
                                                        is_training=1,
                                                        causality=False, 
                                                        scope="self_attention_1")
    multi_att2, _ =  multihead_attention(queries=X2, keys=X2, wkeys = wkeys2, num_units=n_input, 
                                                        num_heads=8, 
                                                        dropout_rate=0.9,
                                                        is_training=1,
                                                        causality=False, 
                                                        scope="self_attention_2")
    multi_att3, _ =  multihead_attention(queries=X3, keys=X3, wkeys = wkeys3, num_units=n_input, 
                                                        num_heads=8, 
                                                        dropout_rate=0.9,
                                                        is_training=1,
                                                        causality=False, 
                                                        scope="self_attention_3")
    
    multi_res = multi_att + multi_att2 + multi_att3 
    
    with tf.variable_scope("fc_1") as fc_1:
            fc_1 = tf.contrib.layers.fully_connected(tf.reshape(multi_res,(n_frames,1*n_hidden)), 3, activation_fn=tf.nn.relu)
            fc_1_drop = tf.nn.dropout(fc_1, 0.8)
            fc_1_variables =  [v for v in tf.global_variables() if v.name.startswith(fc_1.name)]        
    
    for i in range(n_frames):                  
        with tf.variable_scope('model',reuse=tf.AUTO_REUSE):
            
            mul1 =  tf.reshape(multi_att[0,i,:],(1,1*n_hidden))
            mul2 =  tf.reshape(multi_att2[0,i,:],(1,1*n_hidden))
            mul3 =  tf.reshape(multi_att3[0,i,:],(1,1*n_hidden))
            m1   =  fc_1_drop[i,0]
            m2   =  fc_1_drop[i,1]
            m3   =  fc_1_drop[i,2]
            m1   =  tf.sigmoid(m1)
            m2   =  tf.sigmoid(m2)
            m3   =  tf.sigmoid(m3)
            multi_ahead = tf.multiply(m1,mul1) + tf.multiply(m2,mul2) + tf.multiply(m3, mul3)
            
#             multi_ahead = tf.reshape(multi_res[0,i,:],(1,n_hidden))
            with tf.variable_scope("fc_m") as fc_m:
                fc_m = tf.contrib.layers.fully_connected(multi_ahead,n_hidden, activation_fn=tf.nn.relu)
                fc_m_drop = tf.nn.dropout(fc_m, 0.8)
                fc_m_variables =  [v for v in tf.global_variables() if v.name.startswith(fc_m.name)]   
            
            with tf.variable_scope("fc_m2") as fc_m2:
                fc_m2 = tf.contrib.layers.fully_connected(fc_m_drop,n_hidden, activation_fn=tf.nn.relu)
                fc_m2_drop = tf.nn.dropout(fc_m2, 0.8)
                fc_m_variables =  [v for v in tf.global_variables() if v.name.startswith(fc_m.name)]

            # FC to output classification
            pred = tf.matmul(fc_m2_drop , weights['out']) + biases['out']

            # save the predict of each time step
            if i == 0:
                soft_pred = tf.reshape((tf.nn.softmax(pred)),(1,n_classes))
 
            else:
                temp_soft_pred = tf.reshape((tf.nn.softmax(pred)),(1,n_classes))
                soft_pred   = tf.concat([soft_pred,temp_soft_pred],axis =0)



            # negative example
            yc = tf.reshape(y[0,:],(1,n_classes))
            neg_loss = tf.nn.softmax_cross_entropy_with_logits(labels = yc,logits = pred) # Softmax loss
            temp_loss = tf.reduce_mean(neg_loss)
            loss = tf.add(loss, temp_loss)
            #loss = tf.add(loss,reg_term)

      
    # Define loss and optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss/n_frames) # Adam Optimizer

    return x,x2,x3,y,optimizer,loss,soft_pred,fc_m_variables,multi_att,matt

In [8]:
tf.reset_default_graph()
x,x2,x3,y,optimizer,loss,soft_pred,fc_m_variables,multi_att,matt= build_model()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True,gpu_options=gpu_options))
# mkdir folder for saving model
if os.path.isdir(save_path) == False:
        os.mkdir(save_path)
# Initializing the variables
init = tf.global_variables_initializer()
# Launch the graph
sess.run(init)
saver = tf.train.Saver()
#saver = tf.train.import_meta_graph(save_path +'model-25.meta')
#saver.restore(sess,save_path+'model-25')
epoch = 1
epoch_loss = np.zeros((train_num,1),dtype = float)
n_batchs = np.arange(1,train_num+1)
np.random.shuffle(n_batchs)
tStart_epoch = time.time()
matt.get_shape

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



<bound method Tensor.get_shape of <tf.Tensor 'self_attention_1/Reshape_4:0' shape=(8, 32, 32) dtype=float32>>

In [9]:
# Keep training until reach max iterations
# start training
for epoch in range(n_epochs):
#  chose batch randomly
    epoch_loss = np.zeros((train_num,1),dtype = float)
    n_batchs = np.arange(1,train_num+1)
    np.random.shuffle(n_batchs)
    tStart_epoch = time.time()
    k = 0
    for batch in n_batchs:
        
        # load features and labels
        file_name    = video_list['video'][0][batch][0][0]
        batch_data   = np.load(train_path   + file_name +'.npy').item()
        batch_data2  = np.load(train_path_2 + file_name +'.npy').item()
        batch_data3  = np.load(train_path_3 + file_name +'.npy').item()
        batch_xs     = batch_data['feat']
        batch_xs2    = batch_data2['feat']
        batch_xs3    = batch_data3['feat']
        
        batch_ys     = batch_data['label']
        totalframe   = batch_xs.shape
        totalframe   = totalframe[0]
        
        # reshape input batch 
        batch_xc  = np.zeros((1,n_frames,batch_xs.shape[1]))
        batch_xc2 = np.zeros((1,n_frames,batch_xs.shape[1]))
        batch_xc3 = np.zeros((1,n_frames,batch_xs.shape[1]))
        batch_yc  = np.zeros((1,n_frames,n_classes))                   
        batch_xc[0,:,:]  = batch_xs
        batch_xc2[0,:,:] = batch_xs2
        batch_xc3[0,:,:] = batch_xs3
        
        # feed-forward
        _,batch_loss = sess.run([optimizer,loss], feed_dict={x: batch_xc, x2: batch_xc2, x3: batch_xc3, y: batch_ys})

        epoch_loss[batch-1] = batch_loss
        # Debug per batch
        k = k + 1

    # print per epoch
    print ("Epoch:", epoch+1, " done. Loss:", np.mean(epoch_loss))
    tStop_epoch = time.time()
    print ("Epoch Time Cost:", round(tStop_epoch - tStart_epoch,2), "s")
    sys.stdout.flush()
    if (epoch+1) %10 == 0:
        saver.save(sess,save_path+"model", global_step = epoch+1)
print ("Optimization Finished!")
saver.save(sess, save_path+"final_model")

('Epoch:', 1, ' done. Loss:', 20.96615699623896)
('Epoch Time Cost:', 56.34, 's')
('Epoch:', 2, ' done. Loss:', 2.829602699536126)
('Epoch Time Cost:', 55.33, 's')
('Epoch:', 3, ' done. Loss:', 1.1719480908727917)
('Epoch Time Cost:', 55.63, 's')
('Epoch:', 4, ' done. Loss:', 1.435274453438173)
('Epoch Time Cost:', 55.32, 's')
('Epoch:', 5, ' done. Loss:', 0.618251042745361)
('Epoch Time Cost:', 55.52, 's')
('Epoch:', 6, ' done. Loss:', 0.9216691430704859)
('Epoch Time Cost:', 55.49, 's')
('Epoch:', 7, ' done. Loss:', 0.6663288922501013)
('Epoch Time Cost:', 55.54, 's')
('Epoch:', 8, ' done. Loss:', 0.6265547199128119)
('Epoch Time Cost:', 55.77, 's')
('Epoch:', 9, ' done. Loss:', 0.04991936112461406)
('Epoch Time Cost:', 55.13, 's')
('Epoch:', 10, ' done. Loss:', 0.826758643872159)
('Epoch Time Cost:', 55.66, 's')
('Epoch:', 11, ' done. Loss:', 0.7833805597700765)
('Epoch Time Cost:', 55.7, 's')
('Epoch:', 12, ' done. Loss:', 0.4710083683045031)
('Epoch Time Cost:', 56.04, 's')
('Epoc

('Epoch:', 98, ' done. Loss:', 0.35051129609369575)
('Epoch Time Cost:', 55.75, 's')
('Epoch:', 99, ' done. Loss:', 0.0007176268839978079)
('Epoch Time Cost:', 55.76, 's')
('Epoch:', 100, ' done. Loss:', 0.00024458927506541974)
('Epoch Time Cost:', 55.68, 's')
Optimization Finished!


'./multi_model/ixmas_fusion/final_model'

In [10]:
tf.reset_default_graph()
att_res = {}
x,x2,x3,y,optimizer,loss,soft_pred,fc_m_variables,multi_att,matt= build_model()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True,gpu_options=gpu_options))
init = tf.global_variables_initializer()
sess.run(init)

saver = tf.train.Saver()

# restore model
print(default_model_path)
saver.restore(sess, default_model_path)
top1_list = []
for num_batch in range(0,test_num+1): # 0, test_num = 0 : test_num-1
    file_name = test_list['video'][0][num_batch][0][0]
    test_all_data = np.load(test_path + file_name+'.npy').item()
    test_all_data2 = np.load(test_path_2 + file_name+'.npy').item()
    test_all_data3 = np.load(test_path_3 + file_name+'.npy').item()
    print(file_name)
    test_data = test_all_data['feat']
    test_data2 = test_all_data2['feat']
    test_data3 = test_all_data3['feat']
   
    #test_data_final = np.concatenate ((test_data,test_data_2),axis = 2)
    test_labels = test_all_data['label']
        
    totalframe = test_data.shape
    totalframe = totalframe[0]
    att_res['name'] = file_name
    #print(totalframe)
    pred_list = np.zeros((n_frames,n_classes))
    batch_xc  = np.zeros((1,n_frames,test_data.shape[1]))
    batch_xc2 = np.zeros((1,n_frames,test_data.shape[1]))
    batch_xc3 = np.zeros((1,n_frames,test_data.shape[1]))
    batch_xc[0,:,:] = test_data
    batch_xc2[0,:,:] = test_data2
    batch_xc3[0,:,:] = test_data3
                 
    [pred,matte] = sess.run([soft_pred,matt], feed_dict={x: batch_xc, x2: batch_xc2, x3: batch_xc3, y:test_labels})
   
    pred_list    = pred
    matt_list    = matte
    

    label_index  = np.where(test_labels==1)[1][0]
    
    avg_pre = np.mean(pred_list, axis=0).tolist()
    top1 = (avg_pre.index(max(avg_pre))==label_index)
    top1_list.append(top1)
    att_res['pred_list'] = avg_pre
    att_res['label'] = test_labels 
    att_res['matt'] = matt_list
  
    print('video number:' ,num_batch, ' of ', test_num)
    print('current mAP = ',np.mean(top1_list))
    
    sio.savemat(output_path+file_name+'.mat',att_res, do_compression=False)

/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


./multi_model/ixmas_fusion/final_model
INFO:tensorflow:Restoring parameters from ./multi_model/ixmas_fusion/final_model
clare2_01_check-watch_cam0_frames_0030_0103
('video number:', 0, ' of ', 179)
('current mAP = ', 1.0)
amel2_01_check-watch_cam1_frames_0076_0123
('video number:', 1, ' of ', 179)
('current mAP = ', 1.0)
hedlena2_01_check-watch_cam2_frames_0052_0122
('video number:', 2, ' of ', 179)
('current mAP = ', 1.0)
nicolas1_01_check-watch_cam2_frames_0050_0119
('video number:', 3, ' of ', 179)
('current mAP = ', 1.0)
andreas2_01_check-watch_cam1_frames_0043_0107
('video number:', 4, ' of ', 179)
('current mAP = ', 1.0)
florian3_01_check-watch_cam4_frames_0040_0090
('video number:', 5, ' of ', 179)
('current mAP = ', 1.0)
clare1_01_check-watch_cam0_frames_0112_0182
('video number:', 6, ' of ', 179)
('current mAP = ', 0.8571428571428571)
chiara2_01_check-watch_cam3_frames_0033_0087
('video number:', 7, ' of ', 179)
('current mAP = ', 0.875)
hedlena1_01_check-watch_cam4_frames_005

alba3_05_get-up_cam4_frames_0348_0400
('video number:', 72, ' of ', 179)
('current mAP = ', 0.9726027397260274)
chiara3_05_get-up_cam4_frames_0293_0386
('video number:', 73, ' of ', 179)
('current mAP = ', 0.972972972972973)
julien1_05_get-up_cam3_frames_0316_0403
('video number:', 74, ' of ', 179)
('current mAP = ', 0.9733333333333334)
hedlena1_06_turn-around_cam3_frames_0501_0564
('video number:', 75, ' of ', 179)
('current mAP = ', 0.9736842105263158)
chiara1_06_turn-around_cam0_frames_0439_0519
('video number:', 76, ' of ', 179)
('current mAP = ', 0.974025974025974)
chiara2_06_turn-around_cam1_frames_0384_0453
('video number:', 77, ' of ', 179)
('current mAP = ', 0.9743589743589743)
daniel2_06_turn-around_cam1_frames_0396_0472
('video number:', 78, ' of ', 179)
('current mAP = ', 0.9746835443037974)
amel2_06_turn-around_cam1_frames_0474_0549
('video number:', 79, ' of ', 179)
('current mAP = ', 0.975)
amel2_06_turn-around_cam3_frames_0474_0549
('video number:', 80, ' of ', 179)
('c

('video number:', 149, ' of ', 179)
('current mAP = ', 0.98)
clare2_11_point_cam0_frames_1089_1146
('video number:', 150, ' of ', 179)
('current mAP = ', 0.9801324503311258)
clare2_11_point_cam3_frames_1089_1146
('video number:', 151, ' of ', 179)
('current mAP = ', 0.9802631578947368)
alba1_11_point_cam4_frames_0872_0967
('video number:', 152, ' of ', 179)
('current mAP = ', 0.9803921568627451)
florian2_11_point_cam3_frames_0839_0888
('video number:', 153, ' of ', 179)
('current mAP = ', 0.9805194805194806)
nicolas1_11_point_cam4_frames_1009_1056
('video number:', 154, ' of ', 179)
('current mAP = ', 0.9741935483870968)
hedlena1_11_point_cam4_frames_1007_1069
('video number:', 155, ' of ', 179)
('current mAP = ', 0.9743589743589743)
daniel3_11_point_cam0_frames_0795_0836
('video number:', 156, ' of ', 179)
('current mAP = ', 0.9745222929936306)
andreas1_11_point_cam4_frames_1009_1067
('video number:', 157, ' of ', 179)
('current mAP = ', 0.9746835443037974)
julien3_11_point_cam4_frame

In [11]:
test_labels

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [12]:
np.where(test_labels==1)[1][0]

11

In [13]:
sum(top1_list)

176